In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy import stats
import matplotlib as plt

In [2]:
data=pd.read_csv('Incidents_Responded_to_by_Fire_Companies.csv')

C:\Miniconda2\envs\tensorflow_env\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (1,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
columns=pd.read_excel('OPEN_DATA_FIRE_INCIDENTS_FILE_DESCRIPTION.xls')

In [4]:
data.head()

,IM_INCIDENT_KEY,FIRE_BOX,INCIDENT_TYPE_DESC,INCIDENT_DATE_TIME,ARRIVAL_DATE_TIME,UNITS_ONSCENE,LAST_UNIT_CLEARED_DATE_TIME,HIGHEST_LEVEL_DESC,TOTAL_INCIDENT_DURATION,ACTION_TAKEN1_DESC,...,ZIP_CODE,BOROUGH_DESC,FLOOR,CO_DETECTOR_PRESENT_DESC,FIRE_ORIGIN_BELOW_GRADE_FLAG,STORY_FIRE_ORIGIN_COUNT,FIRE_SPREAD_DESC,DETECTOR_PRESENCE_DESC,AES_PRESENCE_DESC,STANDPIPE_SYS_PRESENT_FLAG
0,55672688,2147,"300 - Rescue, EMS incident, other",01/01/2013 12:00:20 AM,01/01/2013 12:14:23 AM,1.0,01/01/2013 12:20:06 AM,"1 - More than initial alarm, less than Signal 7-5",1186.0,"00 - Action taken, other",...,10454,2 - Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,55672692,818,735A - Unwarranted alarm/defective condition o...,01/01/2013 12:00:37 AM,01/01/2013 12:09:03 AM,3.0,01/01/2013 12:30:06 AM,"1 - More than initial alarm, less than Signal 7-5",1769.0,86 - Investigate,...,10036,1 - Manhattan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55672693,9656,"300 - Rescue, EMS incident, other",01/01/2013 12:01:17 AM,01/01/2013 12:04:55 AM,1.0,01/01/2013 12:15:18 AM,"1 - More than initial alarm, less than Signal 7-5",841.0,"00 - Action taken, other",...,11418,5 - Queens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55672695,7412,412 - Gas leak (natural gas or LPG),01/01/2013 12:02:32 AM,01/01/2013 12:07:48 AM,4.0,01/01/2013 12:40:11 AM,"1 - More than initial alarm, less than Signal 7-5",2259.0,44 - Hazardous materials leak control & contai...,...,11103,5 - Queens,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55672697,4019,735A - Unwarranted alarm/defective condition o...,01/01/2013 12:01:49 AM,01/01/2013 12:06:27 AM,6.0,01/01/2013 12:24:56 AM,"1 - More than initial alarm, less than Signal 7-5",1387.0,86 - Investigate,...,11385,5 - Queens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#What proportion of FDNY responses in this dataset correspond to the most common type of incident?
Fraction_most_common_incident=data['INCIDENT_TYPE_DESC'].value_counts(normalize=True).max()
print ('%.10f'%Fraction_most_common_incident)

0.3614828304


In [6]:
#What is the ratio of the average number of units that arrive to a scene
#of an incident classified as '111 - Building fire' to the number that arrive
#for '651 - Smoke scare, odor of smoke'?
Avg_OnSceneUnits_Code111=data['UNITS_ONSCENE'][data['INCIDENT_TYPE_DESC']=='111 - Building fire'].mean()
Avg_OnSceneUnits_Code651=data['UNITS_ONSCENE'][data['INCIDENT_TYPE_DESC']=='651 - Smoke scare, odor of smoke'].mean()
Ratio_111_to_651=Avg_OnSceneUnits_Code111/Avg_OnSceneUnits_Code651
print ('%.10f'%Ratio_111_to_651)

2.7597595140


In [7]:
#How many times more likely is an incident in Staten Island a false call compared to in Manhattan? 
#The answer should be the ratio of Staten Island false call rate to Manhattan false call rate.
#A false call is an incident for which 'INCIDENT_TYPE_DESC' is '710 - Malicious, mischievous false call, other'.
Boroughs=data.BOROUGH_DESC.unique()
Manhattan_incidents=data.INCIDENT_TYPE_DESC[data.BOROUGH_DESC=='1 - Manhattan'].value_counts(normalize=True)
Staten_island_incidents=data.INCIDENT_TYPE_DESC[data.BOROUGH_DESC=='3 - Staten Island'].value_counts(normalize=True)
SI_by_MH_FalseCallRate=Staten_island_incidents['710 - Malicious, mischievous false call, other']/Manhattan_incidents['710 - Malicious, mischievous false call, other']
print ('%.10f'%SI_by_MH_FalseCallRate)

1.6243819472


In [8]:
#Check the distribution of the number of minutes it takes between the time a '111 - Building fire'
#incident has been logged into the Computer Aided Dispatch system and the time at which the first 
#unit arrives on scene. What is the third quartile of that distribution. Note: the number of minutes
#can be fractional (ie, do not round).
Incident_time_Code111=pd.to_datetime(data['INCIDENT_DATE_TIME'][data['INCIDENT_TYPE_DESC']=='111 - Building fire'])
Arrival_time_Code111=pd.to_datetime(data['ARRIVAL_DATE_TIME'][data['INCIDENT_TYPE_DESC']=='111 - Building fire'])
Response_time_Code111=(Arrival_time_Code111-Incident_time_Code111)
Q3_Response_time_Code111=Response_time_Code111.quantile(0.75).seconds/60
print ('%.10f'%Q3_Response_time_Code111)

4.1500000000


In [9]:
#Compute what proportion of all incidents are cooking fires for every hour of the day by
#normalizing the number of cooking fires in a given hour by the total number of incidents that occured in that hour. 
#Find the hour of the day that has the highest proportion of cooking fires and submit that proportion of cooking fires.
#A cooking fire is an incident for which 'INCIDENT_TYPE_DESC' is '113 - Cooking fire, confined to container'. 
#Note: round incident times down. For example, if an incident occured at 22:55 it occured in hour 22.
def Hour(Time):
    return Time.hour
data['INCIDENT_HOUR']=pd.to_datetime(data['INCIDENT_DATE_TIME']).apply(Hour)

In [10]:
IncidentCount_Hourwise=data['INCIDENT_HOUR'].value_counts()
IncidentCount_Hourwise_Code113=data['INCIDENT_HOUR'][data['INCIDENT_TYPE_DESC']=='113 - Cooking fire, confined to container'].value_counts()

In [11]:
IncidentCount=pd.DataFrame(IncidentCount_Hourwise_Code113).rename(columns={'INCIDENT_HOUR':'CODE113_INCIDENTCOUNT_HOURWISE'})
IncidentCount=IncidentCount.join(IncidentCount_Hourwise).rename(columns={'INCIDENT_HOUR':'INCIDENTCOUNT_HOURWISE'})
IncidentCount['Normalized_Code113']=IncidentCount['CODE113_INCIDENTCOUNT_HOURWISE']/IncidentCount['INCIDENTCOUNT_HOURWISE']
MaxProp_Code113=IncidentCount['Normalized_Code113'].max()
print ('%.10f'%MaxProp_Code113)

0.0508468245


In [12]:
#What is the coefficient of determination (R squared) between the number of residents at each zip code
#and the number of inicidents whose type is classified as '111 - Building fire' at each of those zip codes.
#Note: The 2010 US Census population by zip code dataset should be downloaded from here. 
#You will need to use both the FDNY responses and the US Census dataset. 
#Ignore zip codes that do not appear in the census table.
ZipCode_data=pd.read_csv('2010+Census+Population+By+Zipcode+(ZCTA).csv',index_col='Zip Code ZCTA')
Code111_zipcode=data['ZIP_CODE'][data['INCIDENT_TYPE_DESC']=='111 - Building fire'].value_counts()
ZipCode_data_final=ZipCode_data.join(Code111_zipcode,how='inner').rename(columns={'ZIP_CODE':'CODE111_Count'})
R_sq=stats.linregress(ZipCode_data_final['2010 Census Population'],ZipCode_data_final['CODE111_Count']).rvalue**2
print ('%.10f'%R_sq)

0.5928733920


In [13]:
#For this question, only consider incidents that have information about whether a CO detector was present or not.
#We are interested in how many times more likely it is that an incident is long when no CO detector is present compared 
#to when a CO detector is present. For events with CO detector and for those without one, compute the proportion of 
#incidents that lasted 20-30, 30-40, 40-50, 50-60, and 60-70 minutes (both interval boundary values included) by dividing
#the number of incidents in each time interval with the total number of incidents. For each bin, compute the ratio of the 
#'CO detector absent' frequency to the 'CO detector present' frequency. Perform a linear regression of this ratio to the
#mid-point of the bins. From this, what is the predicted ratio for events lasting 39 minutes?
CO_detector_duration=data[['TOTAL_INCIDENT_DURATION','CO_DETECTOR_PRESENT_DESC']][data['CO_DETECTOR_PRESENT_DESC'].notna()]
CO_detector_duration['Incident_duration_minutes']=CO_detector_duration['TOTAL_INCIDENT_DURATION']/60
def duration_bin(time_duration):
    if time_duration<20.0:
        return 0
    elif time_duration>=20.0 and time_duration<=30.0:
        return 1
    elif time_duration>=30.0 and time_duration<=40.0:
        return 2
    elif time_duration>=40.0 and time_duration<=50.0:
        return 3
    elif time_duration>=50.0 and time_duration<=60.0:
        return 4
    elif time_duration>=60.0 and time_duration<=70.0:
        return 5
    else:
        return 6
CO_detector_duration['duration_bin']=CO_detector_duration['Incident_duration_minutes'].apply(duration_bin)
CO_detector_present=CO_detector_duration[CO_detector_duration['CO_DETECTOR_PRESENT_DESC']=='Yes']
CO_detector_absent=CO_detector_duration[CO_detector_duration['CO_DETECTOR_PRESENT_DESC']=='No']

In [14]:
CO_detector_bins=pd.DataFrame(CO_detector_present['duration_bin'].value_counts(normalize=True)).rename(columns={'duration_bin':'CO_detector_present_ratio'})
CO_detector_bins=CO_detector_bins.join(CO_detector_absent['duration_bin'].value_counts(normalize=True)).rename(columns={'duration_bin':'CO_detector_absent_ratio'})
CO_detector_bins['ratio']=CO_detector_bins['CO_detector_present_ratio']/CO_detector_bins['CO_detector_absent_ratio']
CO_detector_bins=CO_detector_bins.drop(index=[0,6])
CO_detector_bins['DurationBin_midpoint']=[25,35,45,55,65]
linearModel=stats.linregress(CO_detector_bins['DurationBin_midpoint'],CO_detector_bins['ratio'])
PredictedRatio=linearModel.slope*39+linearModel.intercept
print ('%.10f'%PredictedRatio)

0.8532784850


In [15]:
#Calculate the chi-square test statistic for testing whether an incident is more likely to last longer
#than 60 minutes when CO detector is not present. Again only consider incidents that have information 
#about whether a CO detector was present or not.
CO_detector_duration_incidents=data[['INCIDENT_TYPE_DESC','TOTAL_INCIDENT_DURATION','CO_DETECTOR_PRESENT_DESC']][data['CO_DETECTOR_PRESENT_DESC'].notna()]
CO_detector_duration_incidents['Incident_duration_minutes']=CO_detector_duration['TOTAL_INCIDENT_DURATION']/60
CO_detector_duration_incidents['Incident_Hourplus']=(CO_detector_duration['TOTAL_INCIDENT_DURATION']/60)>60
f_duration_incidents=pd.crosstab(CO_detector_duration_incidents['INCIDENT_TYPE_DESC'],CO_detector_duration_incidents['Incident_Hourplus'])
Chi_sq_test=stats.chi2_contingency(f_duration_incidents)
Chi_sq_test_statistic=Chi_sq_test[0]
print ('%.10f'%Chi_sq_test_statistic)

2972.5302577032
